# <font color='blue'>Data Science Challenge @ ITA 2022</font>
# <font color='blue'>Equipe DIOMGIS</font>

## <font color='blue'>Fase 1</font>

### <font color='blue'>TEMA DO DESAFIO</font>

![title](..\data\image\logo.jpeg)

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.12


In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
#!pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
#!pip install -q -U watermark

# Instala o pacote tensorboard-plugin-profile. 
# Esse pacote é usado para incrementar funcioalidades no Tensorboard.
#!pip install -U tensorboard-plugin-profile

# Instala o pacote scikeras.
# Esse pacote é usado para integrar funcionalidades entre o scikit-learn e keras.
#!pip install scikeras

In [3]:
# Imports
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
# from scikeras.wrappers import KerasClassifier
# from scikeras.wrappers import KerasRegressor
from keras.wrappers.scikit_learn import KerasClassifier
#from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.callbacks import TensorBoard
%load_ext tensorboard
import datetime
from datetime import datetime
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from sklearn.pipeline import Pipeline
from sklearn import datasets # ?
from sklearn.pipeline import Pipeline


from keras.datasets import mnist
from keras.layers import *
from keras.models import *
from time import time

In [ ]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from keras.wrappers.scikit_learn import KerasClassifier

In [4]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Equipe DIOMGIS" --iversions

Author: Equipe DIOMGIS

keras     : 2.10.0
matplotlib: 3.5.1
seaborn   : 0.11.2
sklearn   : 1.1.2
numpy     : 1.22.3
tensorflow: 2.10.0
pandas    : 1.4.2



In [5]:
#Confirme se o TensorFlow pode acessar a GPU.
device_name = tf.test.gpu_device_name()
if not device_name:
    raise SystemError('GPU device not found')
    
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [6]:
seed = 7
np.random.seed(seed)

In [7]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [8]:
def create_model(optimizer="adam", dropout=0.1, init='uniform', nbr_features=2500, dense_nparams=256):
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28)))
    model.add(ense(dense_nparams, activation='relu', kernel_initializer=init,))
    model.add(Dropout(dropout))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=["accuracy"])
              
    return model

In [9]:
kears_estimator = KerasClassifier(build_fn=create_model, verbose=2)

C:\Users\lucas\AppData\Local\Temp\ipykernel_9100\812626670.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  kears_estimator = KerasClassifier(build_fn=create_model, verbose=2)


In [10]:
estimator = Pipeline([("kc", kears_estimator)])

In [11]:
# define the grid search parameters
params_grid = {
    'kc__epochs': [10, 100, ],
    'kc__dense_nparams': [32, 256, 512],
    'kc__init': [ 'uniform', 'zeros', 'normal', ], 
    'kc__batch_size':[2, 16, 32],
    'kc__optimizer':['RMSprop', 'Adam', 'Adamax', 'sgd'],
    'kc__dropout': [0.5, 0.4, 0.3, 0.2, 0.1, 0]
}

In [12]:
save_dir = 'saved_models'

In [13]:
checkpoint = ModelCheckpoint(save_dir, 
                             monitor='val_accuracy',
                             verbose=1, 
                             save_best_only=True,
                             mode='max'
)
    
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")

In [14]:
kfold_splits = 5
grid = GridSearchCV(estimator=estimator,  
                    n_jobs = 4, 
                    verbose= 2,
                    return_train_score = True,
                    cv = kfold_splits,
                    param_grid = params_grid,)

In [15]:
%tensorboard --logdir=logs/scalars

Reusing TensorBoard on port 6006 (pid 2168), started 3:39:40 ago. (Use '!kill 2168' to kill it.)

In [16]:
X = x_train
y = y_train

In [ ]:
grid_result = grid.fit(X, y, callbacks=[tensorboard_callback, checkpoint])

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 5 folds for each of 1296 candidates, totalling 6480 fits


## Carregando o Conjunto de dados

## Análise Exploratória de Dados

### Análise n - XXX

## Pré-Processamento de Dados Para Construção de Modelos de Machine Learning

### Padronização

###  Construção, Treinamento e Avaliação do Modelo 1 com Regressão Linear (Benchmark)

### Avaliação do Modelo

### Métricas

### Resíduos

###  Construção, Treinamento e Avaliação do Modelo n com XXX

## Seleção do Modelo

## Conclusão

# Fim